In [15]:
library(tidyverse)
data("starwars", package = "dplyr")

print(starwars)

# A tibble: 87 × 14
   name     height  mass hair_color skin_color eye_color birth_year sex   gender
   <chr>     <int> <dbl> <chr>      <chr>      <chr>          <dbl> <chr> <chr> 
 1 Luke Sk…    172    77 blond      fair       blue            19   male  mascu…
 2 C-3PO       167    75 NA         gold       yellow         112   none  mascu…
 3 R2-D2        96    32 NA         white, bl… red             33   none  mascu…
 4 Darth V…    202   136 none       white      yellow          41.9 male  mascu…
 5 Leia Or…    150    49 brown      light      brown           19   fema… femin…
 6 Owen La…    178   120 brown, gr… light      blue            52   male  mascu…
 7 Beru Wh…    165    75 brown      light      blue            47   fema… femin…
 8 R5-D4        97    32 NA         white, red red             NA   none  mascu…
 9 Biggs D…    183    84 black      light      brown           24   male  mascu…
10 Obi-Wan…    182    77 auburn, w… fair       blue-gray       57   male  mascu…
# ℹ 77 m

## Q1 — Dataset Inspection and Missing Values (5 points)


In [16]:
glimpse(starwars)

dim(starwars)

starwars %>%
  select(height, mass, homeworld) %>%
  summarise(
    missing_height = sum(is.na(height)),
    missing_mass = sum(is.na(mass)),
    missing_homeworld = sum(is.na(homeworld))
  )

Rows: 87
Columns: 14
$ name       <chr> "Luke Skywalker", "C-3PO", "R2-D2", "Darth Vader", "Leia Or…
$ height     <int> 172, 167, 96, 202, 150, 178, 165, 97, 183, 182, 188, 180, 2…
$ mass       <dbl> 77.0, 75.0, 32.0, 136.0, 49.0, 120.0, 75.0, 32.0, 84.0, 77.…
$ hair_color <chr> "blond", NA, NA, "none", "brown", "brown, grey", "brown", N…
$ skin_color <chr> "fair", "gold", "white, blue", "white", "light", "light", "…
$ eye_color  <chr> "blue", "yellow", "red", "yellow", "brown", "blue", "blue",…
$ birth_year <dbl> 19.0, 112.0, 33.0, 41.9, 19.0, 52.0, 47.0, NA, 24.0, 57.0, …
$ sex        <chr> "male", "none", "none", "male", "female", "male", "female",…
$ gender     <chr> "masculine", "masculine", "masculine", "masculine", "femini…
$ homeworld  <chr> "Tatooine", "Tatooine", "Naboo", "Tatooine", "Alderaan", "T…
$ species    <chr> "Human", "Droid", "Droid", "Human", "Human", "Human", "Huma…
$ films      <list> <"A New Hope", "The Empire Strikes Back", "Return of the J…
$ vehicles   <list>

[1] 87 14

missing_height,missing_mass,missing_homeworld
<int>,<int>,<int>
6,28,10


## Q2 — Create a Wide Summary Table (pivot_wider) (7 points)

In [17]:
starwars_summary <- starwars %>%
  filter(!is.na(species)) %>%
  
  group_by(species, gender) %>%
  
  summarise(mean_height = mean(height, na.rm = TRUE), .groups = "drop") %>%
  
  pivot_wider(
    names_from = gender, 
    values_from = mean_height
  )

print(starwars_summary)

# A tibble: 37 × 3
   species   masculine feminine
   <chr>         <dbl>    <dbl>
 1 Aleena          79        NA
 2 Besalisk       198        NA
 3 Cerean         198        NA
 4 Chagrian       196        NA
 5 Clawdite        NA       168
 6 Droid          140        96
 7 Dug            112        NA
 8 Ewok            88        NA
 9 Geonosian      183        NA
10 Gungan         209.       NA
# ℹ 27 more rows


## Q3 — Convert Wide Table Back to Long Format (pivot_longer) (6 points)

In [18]:
starwars_long <- starwars_summary %>%
  # 1. Convert gender columns back to rows
  pivot_longer(
    cols = -species,             
    names_to = "gender",         
    values_to = "mean_height"    
  ) %>%
  
  drop_na(mean_height)

print(starwars_long)

# A tibble: 41 × 3
   species   gender    mean_height
   <chr>     <chr>           <dbl>
 1 Aleena    masculine          79
 2 Besalisk  masculine         198
 3 Cerean    masculine         198
 4 Chagrian  masculine         196
 5 Clawdite  feminine          168
 6 Droid     masculine         140
 7 Droid     feminine           96
 8 Dug       masculine         112
 9 Ewok      masculine          88
10 Geonosian masculine         183
# ℹ 31 more rows


## Q4 — Create New Variables and Handle Missing Data (6 points)

In [20]:
starwars_final <- starwars %>%
  mutate(
    height_category = case_when(
      height < 170 ~ "short",
      height >= 170 & height < 190 ~ "average", 
      height >= 190 ~ "tall"
    ),
    
    homeworld = replace_na(homeworld, "Unknown")
  )

glimpse(starwars_final)

Rows: 87
Columns: 15
$ name            <chr> "Luke Skywalker", "C-3PO", "R2-D2", "Darth Vader", "Le…
$ height          <int> 172, 167, 96, 202, 150, 178, 165, 97, 183, 182, 188, 1…
$ mass            <dbl> 77.0, 75.0, 32.0, 136.0, 49.0, 120.0, 75.0, 32.0, 84.0…
$ hair_color      <chr> "blond", NA, NA, "none", "brown", "brown, grey", "brow…
$ skin_color      <chr> "fair", "gold", "white, blue", "white", "light", "ligh…
$ eye_color       <chr> "blue", "yellow", "red", "yellow", "brown", "blue", "b…
$ birth_year      <dbl> 19.0, 112.0, 33.0, 41.9, 19.0, 52.0, 47.0, NA, 24.0, 5…
$ sex             <chr> "male", "none", "none", "male", "female", "male", "fem…
$ gender          <chr> "masculine", "masculine", "masculine", "masculine", "f…
$ homeworld       <chr> "Tatooine", "Tatooine", "Naboo", "Tatooine", "Alderaan…
$ species         <chr> "Human", "Droid", "Droid", "Human", "Human", "Human", …
$ films           <list> <"A New Hope", "The Empire Strikes Back", "Return of …
$ vehicles        <

## Q5 — Unnesting and Interpretation (6 points)

In [21]:
top_appearances <- starwars %>%
  select(name, films) %>%
  unnest(films) %>%
  count(name, sort = TRUE) %>%
  head(8)

print(top_appearances)

# A tibble: 8 × 2
  name               n
  <chr>          <int>
1 R2-D2              7
2 C-3PO              6
3 Obi-Wan Kenobi     6
4 Chewbacca          5
5 Leia Organa        5
6 Luke Skywalker     5
7 Palpatine          5
8 Yoda               5


In 2–3 sentences, explain why this data must be converted to long format.

- The films column is originally stored as a "list-column," where a single cell contains a nested list of multiple movie titles for one character. Standard aggregation functions cannot read inside these nested lists, so the data must be converted to long format (unnested) to ensure every movie appearance has its own distinct row. This allows R to treat each film as an individual observation, making it possible to accurately group and count them.